# Loading the libaries

In [43]:
import pandas as pd
import sklearn
import numpy as np
import math
import scipy
import statsmodels.formula.api as sm
import statsmodels.api as smapi
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import StratifiedKFold, KFold

regr = AdaBoostRegressor()
pd.options.display.max_columns = 200
pd.options.display.max_rows = 100

# Loading the datafile with all players' stats and their market value
I am replacing empty fields with nan and then fill these fields and other fields with nan with zeroes (viable as there is no possibility to have a negative rating in any statistic that is relevant for the model). Furthermore, I only include players that played at least one game, have not retired after the worldcup (NachWMWert!=0) and that played at least a half per game(more than 44minutes per Game/Spiel). Then I drop the numbers related to the MatchesPlayed and MinutesPlayed as they could be confounders that just reflect how far a team came in the competition. This is already encoded in the points and stage columns.

In [44]:
spielerdata=pd.read_csv("fifa_centered_clean_reduced_4.csv")

spielerdata.replace('', np.nan, inplace=True)
spielerdata.replace(np.nan, 0, inplace=True)
print(len(spielerdata))
spielerdata["Wertdiffperc"]=spielerdata["WertDifferenz"]/spielerdata["VorWMWert"]
spielerdata=spielerdata[spielerdata["MatchesPlayed_player"]!=0]
spielerdata=spielerdata[spielerdata["NachWMWert"]!=0]
spielerdata=spielerdata[spielerdata["MinPerSpiel"]>44]

spielerdata=spielerdata.drop(["MatchesPlayed_player","MinutesPlayed"], axis=1)

print(len(spielerdata))
spielerdata.head()

609
350


,Country,Spielername_fifa,ClubVorWM,Continent,Confederation,Attempts_player_per_min,AttemptsOnTarget_player_per_min,AttemptsOffTarget_player_per_min,Woodwork_player_per_min,AttemptsInArea_player_per_min,AttemptsOutsideArea_player_per_min,AttemptsInArea_ON_player_per_min,AttemptsOutsideArea_ON_player_per_min,PassesAttempted_player_per_min,PassesCompletedAbs_player_per_min,CrossesAttempted_player_per_min,CrossesCompletedAbs_player_per_min,Corners_player_player_per_min,Throw_Ins_player_per_min,Throw_Ins_Completed_player_per_min,Goals_player_per_min,Assists_player_per_min,Penalties_player_per_min,DistanceCovered_per_min,DistanceCoveredInPossession_per_min,DistanceCoveredNotInPossession_per_min,TopSpeed_player,Yellow_player_per_min,YellowRed_player_per_min,Red_player_per_min,FoulsCommitted_player_per_min,FoulsSuffered_player_per_min,FoulsCausingPenalty_player_per_min,Tackles_player_per_min,TacklesWon_playery_player_per_min,AttemptedClearances_player_per_min,ClearanceRate_player_per_min,Saves_player_per_min,Blocks_player_per_min,RecoveredBalls_per_min,Offsides_player_per_min,SolosIntoArea_player_per_min,LostBalls_per_min,DeliveriesInPenaltyArea_per_min,TacklesSuffered_player_per_min,AttemptsOfTeam_per_min,AttemptsOnTarget_OfTeam_per_min,AttemptsOffTarget_OfTeam_per_min,Woodwork_OfTeam_per_min,AttemptsOnTarget_Inside_OfTeam_per_min,AttemptsOnTarget_Outside_OfTeam_per_min,PassesAttempted_OfTeam_per_min,PassesCompleted_OfTeam_per_min,CrossesAttempted_OfTeam_per_min,Corners_OfTeam_OfTeam_per_min,Throw_Ins_OfTeam_per_min,Assists_OfTeam_per_min,Yellow_OfTeam_per_min,YellowRed_OfTeam_per_min,Red_OfTeam_per_min,FoulsCommitted_OfTeam_per_min,FoulsSuffered_OfTeam_per_min,FoulsCausingPenalty_OfTeam_per_min,Tackles_OfTeam_per_min,TacklesWon_OfTeam_per_min,TacklesSuffered_OfTeam_per_min,AttemptedClearances_OfTeam_per_min,Saves_OfTeam_per_min,SolosIntoArea_OfTeam_per_min,MinPerSpiel,Position,Alter,VorWMWert,NachWMWert,WertDifferenz,Wertjetzt,ligacountry,Teamname,StageCurrent,PointsCurrentGroup,PointsCurrentTotal,StageLast,PointsLastGroup,PointsLastTotal,PointsDiffTotal,PointsDiffGroup,StageDiff,Wertdiffperc
1,ARG,Javier MASCHERANO,FC Barcelona,America,CONMEBOL,0.005556,0.005556,0.000000,0.000000,0.000000,0.005556,0.000000,0.005556,0.869444,0.744444,0.004167,0.000000,0.0,0.004167,0.004167,0.000000,0.000000,0.0,0.112778,0.040972,0.042639,30.3,0.001389,0.0,0.0,0.012500,0.023611,0.0,0.000000,0.013889,0.011111,100.0,0.0,0.008333,0.068056,0.000000,0.000000,0.000000,0.000000,0.000000,0.036508,0.060847,0.000000,0.000000,0.000000,0.109524,0.138934,0.153608,0.016523,0.0,0.011058,0.000000,0.119792,0.0,0.0,0.107813,0.140445,0.0,0.000000,0.0,0.000000,0.078231,0.0,0.000000,102.857143,Defensives Mittelfeld,29.0,15000000.0,15000000.0,0.0,5000000.0,spain,Argentina,5,9,21,3,9,15,6,0,2,0.000000
2,ARG,Pablo ZABALETA,Manchester City,America,CONMEBOL,0.001389,0.000000,0.001389,0.000000,0.001389,0.000000,0.000000,0.000000,0.550000,0.430556,0.015278,0.001389,0.0,0.118056,0.086111,0.000000,0.000000,0.0,0.109861,0.041944,0.039306,29.6,0.000000,0.0,0.0,0.011111,0.006944,0.0,0.000000,0.008333,0.016667,100.0,0.0,0.001389,0.044444,0.000000,0.000000,0.000000,0.000000,0.000000,0.009127,0.000000,0.022817,0.000000,0.000000,0.000000,0.087888,0.088841,0.060584,0.0,0.313301,0.000000,0.000000,0.0,0.0,0.095833,0.041307,0.0,0.000000,0.0,0.000000,0.117347,0.0,0.000000,102.857143,Rechter Verteidiger,29.0,19000000.0,19000000.0,0.0,5000000.0,england,Argentina,5,9,21,3,9,15,6,0,2,0.000000
3,ARG,Ezequiel GARAY,Benfica Lissabon,America,CONMEBOL,0.006944,0.001389,0.005556,0.000000,0.005556,0.001389,0.001389,0.000000,0.538889,0.469444,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.105694,0.036389,0.038333,31.4,0.001389,0.0,0.0,0.011111,0.001389,0.0,0.000000,0.001389,0.029167,86.0,0.0,0.004167,0.072222,0.000000,0.000000,0.000000,0.000000,0.000000,0.045635,0.015212,0.091270,0.000000,0.027381,0.000000,0.086112,0.096865,0.000000,0.0,0.000000,0.000000,0.119792,0.0,0.0,0.095833,0.008

In [45]:
spielerdata.columns.values

array(['Country', 'Spielername_fifa', 'ClubVorWM', 'Continent',
       'Confederation', 'Attempts_player_per_min',
       'AttemptsOnTarget_player_per_min',
       'AttemptsOffTarget_player_per_min', 'Woodwork_player_per_min',
       'AttemptsInArea_player_per_min',
       'AttemptsOutsideArea_player_per_min',
       'AttemptsInArea_ON_player_per_min',
       'AttemptsOutsideArea_ON_player_per_min',
       'PassesAttempted_player_per_min',
       'PassesCompletedAbs_player_per_min',
       'CrossesAttempted_player_per_min',
       'CrossesCompletedAbs_player_per_min',
       'Corners_player_player_per_min', 'Throw_Ins_player_per_min',
       'Throw_Ins_Completed_player_per_min', 'Goals_player_per_min',
       'Assists_player_per_min', 'Penalties_player_per_min',
       'DistanceCovered_per_min', 'DistanceCoveredInPossession_per_min',
       'DistanceCoveredNotInPossession_per_min', 'TopSpeed_player',
       'Yellow_player_per_min', 'YellowRed_player_per_min',
       'Red_player_per_min

In [46]:
spielerdata['Position'].unique()

array(['Defensives Mittelfeld', 'Rechter Verteidiger', 'Innenverteidiger',
       'Mittelstürmer', 'Rechtsaußen', 'Hängende Spitze', 'Torwart',
       'Zentrales Mittelfeld', 'Linkes Mittelfeld', 'Linker Verteidiger',
       'Offensives Mittelfeld', 'Linksaußen', 'Rechtes Mittelfeld'],
      dtype=object)

Only using players that are playing on the defensive end. But we did the exact same procedure also for striker (['Rechtsaußen', 'Mittelstürmer','Hängende Spitze', 'Linksaußen']) or for midfielder (['Defensives Mittelfeld', Zentrales Mittelfeld', 'Linkes Mittelfeld', 'Rechtes Mittelfeld','Offensives Mittelfeld']

In [47]:
positionfilter=["Innenverteidiger","Linker Verteidiger","Rechter Verteidiger"]

spielerdata=spielerdata[spielerdata['Position'].isin(positionfilter)]

print(len(spielerdata))
spielerdata.head()

135


,Country,Spielername_fifa,ClubVorWM,Continent,Confederation,Attempts_player_per_min,AttemptsOnTarget_player_per_min,AttemptsOffTarget_player_per_min,Woodwork_player_per_min,AttemptsInArea_player_per_min,AttemptsOutsideArea_player_per_min,AttemptsInArea_ON_player_per_min,AttemptsOutsideArea_ON_player_per_min,PassesAttempted_player_per_min,PassesCompletedAbs_player_per_min,CrossesAttempted_player_per_min,CrossesCompletedAbs_player_per_min,Corners_player_player_per_min,Throw_Ins_player_per_min,Throw_Ins_Completed_player_per_min,Goals_player_per_min,Assists_player_per_min,Penalties_player_per_min,DistanceCovered_per_min,DistanceCoveredInPossession_per_min,DistanceCoveredNotInPossession_per_min,TopSpeed_player,Yellow_player_per_min,YellowRed_player_per_min,Red_player_per_min,FoulsCommitted_player_per_min,FoulsSuffered_player_per_min,FoulsCausingPenalty_player_per_min,Tackles_player_per_min,TacklesWon_playery_player_per_min,AttemptedClearances_player_per_min,ClearanceRate_player_per_min,Saves_player_per_min,Blocks_player_per_min,RecoveredBalls_per_min,Offsides_player_per_min,SolosIntoArea_player_per_min,LostBalls_per_min,DeliveriesInPenaltyArea_per_min,TacklesSuffered_player_per_min,AttemptsOfTeam_per_min,AttemptsOnTarget_OfTeam_per_min,AttemptsOffTarget_OfTeam_per_min,Woodwork_OfTeam_per_min,AttemptsOnTarget_Inside_OfTeam_per_min,AttemptsOnTarget_Outside_OfTeam_per_min,PassesAttempted_OfTeam_per_min,PassesCompleted_OfTeam_per_min,CrossesAttempted_OfTeam_per_min,Corners_OfTeam_OfTeam_per_min,Throw_Ins_OfTeam_per_min,Assists_OfTeam_per_min,Yellow_OfTeam_per_min,YellowRed_OfTeam_per_min,Red_OfTeam_per_min,FoulsCommitted_OfTeam_per_min,FoulsSuffered_OfTeam_per_min,FoulsCausingPenalty_OfTeam_per_min,Tackles_OfTeam_per_min,TacklesWon_OfTeam_per_min,TacklesSuffered_OfTeam_per_min,AttemptedClearances_OfTeam_per_min,Saves_OfTeam_per_min,SolosIntoArea_OfTeam_per_min,MinPerSpiel,Position,Alter,VorWMWert,NachWMWert,WertDifferenz,Wertjetzt,ligacountry,Teamname,StageCurrent,PointsCurrentGroup,PointsCurrentTotal,StageLast,PointsLastGroup,PointsLastTotal,PointsDiffTotal,PointsDiffGroup,StageDiff,Wertdiffperc
2,ARG,Pablo ZABALETA,Manchester City,America,CONMEBOL,0.001389,0.000000,0.001389,0.0,0.001389,0.000000,0.000000,0.000000,0.550000,0.430556,0.015278,0.001389,0.0,0.118056,0.086111,0.000000,0.0,0.0,0.109861,0.041944,0.039306,29.6,0.000000,0.0,0.0,0.011111,0.006944,0.0,0.000000,0.008333,0.016667,100.0,0.0,0.001389,0.044444,0.0,0.000000,0.000000,0.000000,0.000000,0.009127,0.000000,0.022817,0.0,0.000000,0.000000,0.087888,0.088841,0.060584,0.0,0.313301,0.0,0.000000,0.0,0.0,0.095833,0.041307,0.0,0.000000,0.0,0.000000,0.117347,0.0,0.000000,102.857143,Rechter Verteidiger,29.0,19000000.0,19000000.0,0.0,5000000.0,england,Argentina,5,9,21,3,9,15,6,0,2,0.000000
3,ARG,Ezequiel GARAY,Benfica Lissabon,America,CONMEBOL,0.006944,0.001389,0.005556,0.0,0.005556,0.001389,0.001389,0.000000,0.538889,0.469444,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.105694,0.036389,0.038333,31.4,0.001389,0.0,0.0,0.011111,0.001389,0.0,0.000000,0.001389,0.029167,86.0,0.0,0.004167,0.072222,0.0,0.000000,0.000000,0.000000,0.000000,0.045635,0.015212,0.091270,0.0,0.027381,0.000000,0.086112,0.096865,0.000000,0.0,0.000000,0.0,0.119792,0.0,0.0,0.095833,0.008261,0.0,0.000000,0.0,0.000000,0.205357,0.0,0.000000,102.857143,Innenverteidiger,27.0,20000000.0,20000000.0,0.0,15000000.0,portugal,Argentina,5,9,21,3,9,15,6,0,2,0.000000
4,ARG,Marcos ROJO,Sporting Lissabon,America,CONMEBOL,0.013008,0.006504,0.006504,0.0,0.008130,0.004878,0.004878,0.001626,0.595122,0.429268,0.042276,0.008130,0.0,0.082927,0.066667,0.001626,0.0,0.0,0.118211,0.045366,0.041463,31.8,0.003252,0.0,0.0,0.011382,0.009756,0.0,0.019512,0.008130,0.019512,84.0,0.0,0.003252,0.074797,0.0,0.001626,0.112195,0.011382,0.013008,0.085482,0.071235,0.106852,0.0,0.096167,0.032056,0.095098,0.088575,0.167648,0.0,0.220075,0.0,0.280488,0.0,0.0,0.098171,0.058032,0.0,0.114097,0.0,0.083884,0.137382,0.0,0.035061,87.857143,Innenverteidiger

Now I one-hot encode the categorical variables of continent, confederation of the nationalteam, the players position and also which country his regular club team is based in and then drop the categorical columns.

In [48]:
categoricalcat=["Continent","Confederation","Position","ligacountry"]

In [49]:
onehot=pd.get_dummies(spielerdata[categoricalcat])
new_spielerdata=spielerdata.drop(categoricalcat,axis=1)
new_spielerdata=new_spielerdata.join(onehot)
print(len(new_spielerdata))
new_spielerdata.head()


135


,Country,Spielername_fifa,ClubVorWM,Attempts_player_per_min,AttemptsOnTarget_player_per_min,AttemptsOffTarget_player_per_min,Woodwork_player_per_min,AttemptsInArea_player_per_min,AttemptsOutsideArea_player_per_min,AttemptsInArea_ON_player_per_min,AttemptsOutsideArea_ON_player_per_min,PassesAttempted_player_per_min,PassesCompletedAbs_player_per_min,CrossesAttempted_player_per_min,CrossesCompletedAbs_player_per_min,Corners_player_player_per_min,Throw_Ins_player_per_min,Throw_Ins_Completed_player_per_min,Goals_player_per_min,Assists_player_per_min,Penalties_player_per_min,DistanceCovered_per_min,DistanceCoveredInPossession_per_min,DistanceCoveredNotInPossession_per_min,TopSpeed_player,Yellow_player_per_min,YellowRed_player_per_min,Red_player_per_min,FoulsCommitted_player_per_min,FoulsSuffered_player_per_min,FoulsCausingPenalty_player_per_min,Tackles_player_per_min,TacklesWon_playery_player_per_min,AttemptedClearances_player_per_min,ClearanceRate_player_per_min,Saves_player_per_min,Blocks_player_per_min,RecoveredBalls_per_min,Offsides_player_per_min,SolosIntoArea_player_per_min,LostBalls_per_min,DeliveriesInPenaltyArea_per_min,TacklesSuffered_player_per_min,AttemptsOfTeam_per_min,AttemptsOnTarget_OfTeam_per_min,AttemptsOffTarget_OfTeam_per_min,Woodwork_OfTeam_per_min,AttemptsOnTarget_Inside_OfTeam_per_min,AttemptsOnTarget_Outside_OfTeam_per_min,PassesAttempted_OfTeam_per_min,PassesCompleted_OfTeam_per_min,CrossesAttempted_OfTeam_per_min,Corners_OfTeam_OfTeam_per_min,Throw_Ins_OfTeam_per_min,Assists_OfTeam_per_min,Yellow_OfTeam_per_min,YellowRed_OfTeam_per_min,Red_OfTeam_per_min,FoulsCommitted_OfTeam_per_min,FoulsSuffered_OfTeam_per_min,FoulsCausingPenalty_OfTeam_per_min,Tackles_OfTeam_per_min,TacklesWon_OfTeam_per_min,TacklesSuffered_OfTeam_per_min,AttemptedClearances_OfTeam_per_min,Saves_OfTeam_per_min,SolosIntoArea_OfTeam_per_min,MinPerSpiel,Alter,VorWMWert,NachWMWert,WertDifferenz,Wertjetzt,Teamname,StageCurrent,PointsCurrentGroup,PointsCurrentTotal,StageLast,PointsLastGroup,PointsLastTotal,PointsDiffTotal,PointsDiffGroup,StageDiff,Wertdiffperc,Continent_Africa,Continent_America,Continent_Asia,Continent_Australia,Continent_Europe,Confederation_AFC,Confederation_CAF,Confederation_CONCACAF,Confederation_CONMEBOL,Confederation_UEFA,Position_Innenverteidiger,Position_Linker Verteidiger,Position_Rechter Verteidiger,ligacountry_australia,ligacountry_belgium,ligacountry_brazil,ligacountry_china,ligacountry_costa rica,ligacountry_ecuador,ligacountry_england,ligacountry_france,ligacountry_germany,ligacountry_greece,ligacountry_honduras,ligacountry_iran,ligacountry_israeli,ligacountry_italy,ligacountry_korea,ligacountry_mexico,ligacountry_netherlands,ligacountry_norway,ligacountry_portugal,ligacountry_qatar,ligacountry_russia,ligacountry_scotland,ligacountry_spain,ligacountry_switzerland,ligacountry_tunisia,ligacountry_turkey,ligacountry_ukraine,ligacountry_unitedarabianemirtaes,ligacountry_usa
2,ARG,Pablo ZABALETA,Manchester City,0.001389,0.000000,0.001389,0.0,0.001389,0.000000,0.000000,0.000000,0.550000,0.430556,0.015278,0.001389,0.0,0.118056,0.086111,0.000000,0.0,0.0,0.109861,0.041944,0.039306,29.6,0.000000,0.0,0.0,0.011111,0.006944,0.0,0.000000,0.008333,0.016667,100.0,0.0,0.001389,0.044444,0.0,0.000000,0.000000,0.000000,0.000000,0.009127,0.000000,0.022817,0.0,0.000000,0.000000,0.087888,0.088841,0.060584,0.0,0.313301,0.0,0.000000,0.0,0.0,0.095833,0.041307,0.0,0.000000,0.0,0.000000,0.117347,0.0,0.000000,102.857143,29.0,19000000.0,19000000.0,0.0,5000000.0,Argentina,5,9,21,3,9,15,6,0,2,0.000000,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ARG,Ezequiel GARAY,Benfica Lissabon,0.006944,0.001389,0.005556,0.0,0.005556,0.001389,0.001389,0.000000,0.538889,0.469444,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.105694,0.036389,0.038333,31.4,0.001389,0.0,0.0,0.011111,0.001389,0.0,0.000000,0.001389,0.029167,86.0,0.0,0.004167,0.072222,0.0,0.000000,0.000000,0.000000,0.000000,0.045635,0.015212,0.09

# 10-Fold Split
We run a KFold Split with the fixed random state=90 that I picked randomly, to reproduce similar numbers all the time. Them we run the train and test the model ten times and keep on adding the predicted market value and the real market value to an array. Also we keep on collecting the importance values of each feature through the ten folds to find out what was the most important across the ten runs

In [50]:
#Ein DF für unsere Results
totaldf=pd.DataFrame()

#Wir wollen zehn Splits
kfold = KFold(n_splits=10, shuffle=True, random_state=90)
#Und resetted das Array nach allen Filterungen
new_spielerdata=new_spielerdata.reset_index(drop=True)

Y=new_spielerdata["NachWMWert"]

X=new_spielerdata.drop(["NachWMWert"], axis=1)

droplist=["Spielername_fifa","ClubVorWM","WertDifferenz","Wertdiffperc","VorWMWert","Wertjetzt","Country","Teamname"]
    
columdf=X.drop(droplist, axis=1)

columnames=(list(columdf.columns.values))
print(columnames)
importancedf = pd.DataFrame({'columnames':columnames})

a=0
#Wir generieren arrays mit den Indices für die Splits
for train_index, test_index in kfold.split(X, Y):
    a+=1
    #Und für das DF konvertieren das Array zu einer List
    test_indexlist=test_index.tolist()
    
    #Wir nehmen nur die Reihen aus dem DF, die wir durch die test_indexlist definiert haben, um sie später zu nutzen 
    testdata_ML_DF=(new_spielerdata.iloc[test_indexlist,:])
    
    #DF für die Dependent Variable
    Y=new_spielerdata["NachWMWert"]

    #Alle anderen Variablen für den Fit, weiterhin in DF-Form
    X=new_spielerdata.drop(["NachWMWert"], axis=1)
    
    #Jetzt schmeissen wir die Categorical Vars und alles andere raus
    droplist=["Spielername_fifa","ClubVorWM","WertDifferenz","Wertdiffperc","VorWMWert","Wertjetzt","Country","Teamname"]
    
    X=X.drop(droplist, axis=1)
    print(len(X))

    
    #Wir wandeln die DFs in Numpy Arrays um
    Y_array = np.array(Y)
    X_array = np.array(X)
    #Filtern jetzt gemäss unseres Splits
    traindata_ML_X, testdata_ML_X = X_array[train_index], X_array[test_index]
    traindata_ML_Y, testdata_ML_Y = Y_array[train_index], Y_array[test_index]


    #Machen unseren Model-Fit
    regr.fit(traindata_ML_X, traindata_ML_Y)
    
    #Wir ziehen uns alle Columns vom DF X, das wir umgewandelt in Array-Form eingespeist haben
    columnames=(list(X.columns.values))
    importances=(regr.feature_importances_.tolist())
    coldf = pd.DataFrame({'columnames':columnames,'importances': importances})
    coldf=coldf.sort_values(by="columnames", ascending=False)
    print(coldf.head(10))
    importancedf=importancedf.join(coldf["importances"], rsuffix=a)
    print(importancedf.head(10))

    #Nun machen wir basierend auf unserem Model unsere Vorhersagen
    prediction=(regr.predict(testdata_ML_X).tolist())
    #Und holen uns auch unsere echten Werte
    reality=(testdata_ML_Y.tolist())
    
    #Jetzt ziehen wir uns das ganz am Anfang gespeicherte DF und fügen unsere prediction und reality hinzu
    testdata_ML_DF=testdata_ML_DF.reset_index(drop=True)
    df=testdata_ML_DF
    df['prediction']=prediction
    df['reality']=reality
    
    #Fügen das aktuelle DF hinzu
    totaldf=totaldf.append(df)
totaldf

['Attempts_player_per_min', 'AttemptsOnTarget_player_per_min', 'AttemptsOffTarget_player_per_min', 'Woodwork_player_per_min', 'AttemptsInArea_player_per_min', 'AttemptsOutsideArea_player_per_min', 'AttemptsInArea_ON_player_per_min', 'AttemptsOutsideArea_ON_player_per_min', 'PassesAttempted_player_per_min', 'PassesCompletedAbs_player_per_min', 'CrossesAttempted_player_per_min', 'CrossesCompletedAbs_player_per_min', 'Corners_player_player_per_min', 'Throw_Ins_player_per_min', 'Throw_Ins_Completed_player_per_min', 'Goals_player_per_min', 'Assists_player_per_min', 'Penalties_player_per_min', 'DistanceCovered_per_min', 'DistanceCoveredInPossession_per_min', 'DistanceCoveredNotInPossession_per_min', 'TopSpeed_player', 'Yellow_player_per_min', 'YellowRed_player_per_min', 'Red_player_per_min', 'FoulsCommitted_player_per_min', 'FoulsSuffered_player_per_min', 'FoulsCausingPenalty_player_per_min', 'Tackles_player_per_min', 'TacklesWon_playery_player_per_min', 'AttemptedClearances_player_per_min',

                            columnames  importances
116                    ligacountry_usa     0.000000
115  ligacountry_unitedarabianemirtaes     0.000000
114                ligacountry_ukraine     0.000000
113                 ligacountry_turkey     0.000000
112                ligacountry_tunisia     0.000000
111            ligacountry_switzerland     0.000000
110                  ligacountry_spain     0.023036
109               ligacountry_scotland     0.000000
108                 ligacountry_russia     0.000000
107                  ligacountry_qatar     0.000000
                              columnames  importances  importances2  \
0                Attempts_player_per_min     0.012092      0.011192   
1        AttemptsOnTarget_player_per_min     0.038510      0.045240   
2       AttemptsOffTarget_player_per_min     0.001115      0.012133   
3                Woodwork_player_per_min     0.000000      0.000000   
4          AttemptsInArea_player_per_min     0.020131      0.005879   
5 

                            columnames  importances
116                    ligacountry_usa     0.000000
115  ligacountry_unitedarabianemirtaes     0.000000
114                ligacountry_ukraine     0.000000
113                 ligacountry_turkey     0.000000
112                ligacountry_tunisia     0.000000
111            ligacountry_switzerland     0.000000
110                  ligacountry_spain     0.007519
109               ligacountry_scotland     0.000000
108                 ligacountry_russia     0.000000
107                  ligacountry_qatar     0.000000
                              columnames  importances  importances2  \
0                Attempts_player_per_min     0.012092      0.011192   
1        AttemptsOnTarget_player_per_min     0.038510      0.045240   
2       AttemptsOffTarget_player_per_min     0.001115      0.012133   
3                Woodwork_player_per_min     0.000000      0.000000   
4          AttemptsInArea_player_per_min     0.020131      0.005879   
5 

,Country,Spielername_fifa,ClubVorWM,Attempts_player_per_min,AttemptsOnTarget_player_per_min,AttemptsOffTarget_player_per_min,Woodwork_player_per_min,AttemptsInArea_player_per_min,AttemptsOutsideArea_player_per_min,AttemptsInArea_ON_player_per_min,AttemptsOutsideArea_ON_player_per_min,PassesAttempted_player_per_min,PassesCompletedAbs_player_per_min,CrossesAttempted_player_per_min,CrossesCompletedAbs_player_per_min,Corners_player_player_per_min,Throw_Ins_player_per_min,Throw_Ins_Completed_player_per_min,Goals_player_per_min,Assists_player_per_min,Penalties_player_per_min,DistanceCovered_per_min,DistanceCoveredInPossession_per_min,DistanceCoveredNotInPossession_per_min,TopSpeed_player,Yellow_player_per_min,YellowRed_player_per_min,Red_player_per_min,FoulsCommitted_player_per_min,FoulsSuffered_player_per_min,FoulsCausingPenalty_player_per_min,Tackles_player_per_min,TacklesWon_playery_player_per_min,AttemptedClearances_player_per_min,ClearanceRate_player_per_min,Saves_player_per_min,Blocks_player_per_min,RecoveredBalls_per_min,Offsides_player_per_min,SolosIntoArea_player_per_min,LostBalls_per_min,DeliveriesInPenaltyArea_per_min,TacklesSuffered_player_per_min,AttemptsOfTeam_per_min,AttemptsOnTarget_OfTeam_per_min,AttemptsOffTarget_OfTeam_per_min,Woodwork_OfTeam_per_min,AttemptsOnTarget_Inside_OfTeam_per_min,AttemptsOnTarget_Outside_OfTeam_per_min,PassesAttempted_OfTeam_per_min,PassesCompleted_OfTeam_per_min,CrossesAttempted_OfTeam_per_min,Corners_OfTeam_OfTeam_per_min,Throw_Ins_OfTeam_per_min,Assists_OfTeam_per_min,Yellow_OfTeam_per_min,YellowRed_OfTeam_per_min,Red_OfTeam_per_min,FoulsCommitted_OfTeam_per_min,FoulsSuffered_OfTeam_per_min,FoulsCausingPenalty_OfTeam_per_min,Tackles_OfTeam_per_min,TacklesWon_OfTeam_per_min,TacklesSuffered_OfTeam_per_min,AttemptedClearances_OfTeam_per_min,Saves_OfTeam_per_min,SolosIntoArea_OfTeam_per_min,MinPerSpiel,Alter,VorWMWert,NachWMWert,WertDifferenz,Wertjetzt,Teamname,StageCurrent,PointsCurrentGroup,PointsCurrentTotal,StageLast,PointsLastGroup,PointsLastTotal,PointsDiffTotal,PointsDiffGroup,StageDiff,Wertdiffperc,Continent_Africa,Continent_America,Continent_Asia,Continent_Australia,Continent_Europe,Confederation_AFC,Confederation_CAF,Confederation_CONCACAF,Confederation_CONMEBOL,Confederation_UEFA,Position_Innenverteidiger,Position_Linker Verteidiger,Position_Rechter Verteidiger,ligacountry_australia,ligacountry_belgium,ligacountry_brazil,ligacountry_china,ligacountry_costa rica,ligacountry_ecuador,ligacountry_england,ligacountry_france,ligacountry_germany,ligacountry_greece,ligacountry_honduras,ligacountry_iran,ligacountry_israeli,ligacountry_italy,ligacountry_korea,ligacountry_mexico,ligacountry_netherlands,ligacountry_norway,ligacountry_portugal,ligacountry_qatar,ligacountry_russia,ligacountry_scotland,ligacountry_spain,ligacountry_switzerland,ligacountry_tunisia,ligacountry_turkey,ligacountry_ukraine,ligacountry_unitedarabianemirtaes,ligacountry_usa,prediction,reality
0,COL,Juan ZUNIGA,SSC Neapel,0.005556,0.002778,0.002778,0.000000,0.000000,0.005556,0.000000,0.002778,0.563889,0.416667,0.008333,0.002778,0.000000,0.094444,0.047222,0.000000,0.000000,0.0,0.101389,0.032778,0.033889,30.5,0.000000,0.000000,0.0,0.011111,0.038889,0.000000,0.000000,0.002778,0.008333,100.0,0.0,0.000000,0.047222,0.000000,0.000000,0.000000,0.000000,0.000000,0.043860,0.032895,0.065789,0.000000,0.000000,0.065789,0.122703,0.126947,0.057692,0.000000,0.267296,0.000000,0.000000,0.000000,0.0,0.054945,0.208333,0.000000,0.000000,0.000000e+00,0.000000,0.066964,0.0,0.000000,72.000000,28.0,13000000.0,11000000.0,-2000000.0,500000.0,Colombia,3,9,15,0,0,0,15,9,3,-0.153846,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.483333e+06,11000000.0
1,COL,Cristian ZAPATA,AC Mailand,0.005556,0.002778,0.002778,0.000000,0.005556,0.000000,0.002778,0.000000,0.375000,0.300000,0.000000,0.000000,0.000000,0.011111,0.002778,0.000000,0.000000,0.0,0.102500,0.030278,0.034444,31.4,0.000000,0.000000,0.0,0.011111,0.011111,0

After these runs we check the correlation between predicted market value and true market value with pearson and spearman's r. Also we look at the importance of the features across the ten runs.

In [51]:
finaldf=totaldf.reset_index(drop=True)
print(finaldf['prediction'].corr(finaldf['reality']))
print(scipy.stats.spearmanr(finaldf['reality'],finaldf['prediction']).correlation)

finaldf.to_csv("Predictions_Abwehr_2.csv", index=False)
colums=importancedf["columnames"]
vals=importancedf.mean(axis=1)
df=pd.DataFrame({"columns":colums,"vals":vals})
df=df.sort_values(by="vals", ascending=False)
df

0.6255149043460674
0.6878045830061816


,columns,vals
71,PointsLastTotal,0.076386
20,DistanceCoveredNotInPossession_per_min,0.069643
34,RecoveredBalls_per_min,0.059776
65,Alter,0.041137
68,PointsCurrentTotal,0.039118
9,PassesCompletedAbs_player_per_min,0.035715
84,Confederation_UEFA,0.030562
64,MinPerSpiel,0.028966
42,AttemptsOffTarget_OfTeam_per_min,0.028020
79,Continent_Europe,0.027031


In [52]:
df.to_csv("importance_abwehr2.csv", index=False)

We then create an array only with the name of the player, his team, his market value currently, before the world cup, the difference between before and after the world cup, the real value after the world cup, the predicted market value and lastly the difference between the two

In [53]:
finaldf
justcash=finaldf[["Spielername_fifa","Teamname","Wertjetzt","VorWMWert","WertDifferenz","NachWMWert","prediction",]]
justcash["Differenz_Real_Pred"]=justcash["NachWMWert"]-justcash["prediction"]
#justcash=justcash[(justcash["WertDifferenz"]>1000000)|(justcash["WertDifferenz"]<-1000000)]
justcash=justcash.reset_index(drop=True)
justcash

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Spielername_fifa,Teamname,Wertjetzt,VorWMWert,WertDifferenz,NachWMWert,prediction,Differenz_Real_Pred
0,Juan ZUNIGA,Colombia,500000.0,13000000.0,-2000000.0,11000000.0,6.483333e+06,4.516667e+06
1,Cristian ZAPATA,Colombia,2000000.0,5000000.0,0.0,5000000.0,5.525714e+06,-5.257143e+05
2,Madjid BOUGUERRA,Algeria,0.0,500000.0,700000.0,1200000.0,4.737500e+06,-3.537500e+06
3,Alvaro PEREIRA,Uruguay,1000000.0,6000000.0,-500000.0,5500000.0,8.222222e+06,-2.722222e+06
4,Efe AMBROSE,Nigeria,550000.0,3000000.0,750000.0,3750000.0,4.464815e+06,-7.148148e+05
5,Ricardo Costa,Portugal,250000.0,3000000.0,-1000000.0,2000000.0,6.205556e+06,-4.205556e+06
6,Jalal HOSSEINI,Iran,500000.0,2000000.0,-200000.0,1800000.0,4.107609e+06,-2.307609e+06
7,Danijel Pranjic,Croatia,200000.0,1000000.0,0.0,1000000.0,7.807547e+06,-6.807547e+06
8,Jonathan Mensah,Ghana,800000.0,1250000.0,500000.0,1750000.0,6.483333e+06,-4.733333e+06
9,Ron VLAAR,Netherlands,2000000.0,5000000.0,1500000.0,6500000.0,1.825000e+07,-1.175000e+07


Lastly, we check how many of the predicted top 20 are in the real top 20 market-value wise. 

In [54]:
preddf=justcash.sort_values(by="prediction", ascending=False)
top20_pred=preddf["Spielername_fifa"].head(20).tolist()
realdf=justcash.sort_values(by="NachWMWert", ascending=False)
top20_real=realdf["Spielername_fifa"].head(20).tolist()
len([x for x in top20_pred if x in top20_real])

17

In [55]:
justcash.to_csv("Cash_Abwehr2.csv", index=False)

In [56]:
finaldf.to_csv("Stats_Abwehr2.csv", index=False)
finaldf


,Country,Spielername_fifa,ClubVorWM,Attempts_player_per_min,AttemptsOnTarget_player_per_min,AttemptsOffTarget_player_per_min,Woodwork_player_per_min,AttemptsInArea_player_per_min,AttemptsOutsideArea_player_per_min,AttemptsInArea_ON_player_per_min,AttemptsOutsideArea_ON_player_per_min,PassesAttempted_player_per_min,PassesCompletedAbs_player_per_min,CrossesAttempted_player_per_min,CrossesCompletedAbs_player_per_min,Corners_player_player_per_min,Throw_Ins_player_per_min,Throw_Ins_Completed_player_per_min,Goals_player_per_min,Assists_player_per_min,Penalties_player_per_min,DistanceCovered_per_min,DistanceCoveredInPossession_per_min,DistanceCoveredNotInPossession_per_min,TopSpeed_player,Yellow_player_per_min,YellowRed_player_per_min,Red_player_per_min,FoulsCommitted_player_per_min,FoulsSuffered_player_per_min,FoulsCausingPenalty_player_per_min,Tackles_player_per_min,TacklesWon_playery_player_per_min,AttemptedClearances_player_per_min,ClearanceRate_player_per_min,Saves_player_per_min,Blocks_player_per_min,RecoveredBalls_per_min,Offsides_player_per_min,SolosIntoArea_player_per_min,LostBalls_per_min,DeliveriesInPenaltyArea_per_min,TacklesSuffered_player_per_min,AttemptsOfTeam_per_min,AttemptsOnTarget_OfTeam_per_min,AttemptsOffTarget_OfTeam_per_min,Woodwork_OfTeam_per_min,AttemptsOnTarget_Inside_OfTeam_per_min,AttemptsOnTarget_Outside_OfTeam_per_min,PassesAttempted_OfTeam_per_min,PassesCompleted_OfTeam_per_min,CrossesAttempted_OfTeam_per_min,Corners_OfTeam_OfTeam_per_min,Throw_Ins_OfTeam_per_min,Assists_OfTeam_per_min,Yellow_OfTeam_per_min,YellowRed_OfTeam_per_min,Red_OfTeam_per_min,FoulsCommitted_OfTeam_per_min,FoulsSuffered_OfTeam_per_min,FoulsCausingPenalty_OfTeam_per_min,Tackles_OfTeam_per_min,TacklesWon_OfTeam_per_min,TacklesSuffered_OfTeam_per_min,AttemptedClearances_OfTeam_per_min,Saves_OfTeam_per_min,SolosIntoArea_OfTeam_per_min,MinPerSpiel,Alter,VorWMWert,NachWMWert,WertDifferenz,Wertjetzt,Teamname,StageCurrent,PointsCurrentGroup,PointsCurrentTotal,StageLast,PointsLastGroup,PointsLastTotal,PointsDiffTotal,PointsDiffGroup,StageDiff,Wertdiffperc,Continent_Africa,Continent_America,Continent_Asia,Continent_Australia,Continent_Europe,Confederation_AFC,Confederation_CAF,Confederation_CONCACAF,Confederation_CONMEBOL,Confederation_UEFA,Position_Innenverteidiger,Position_Linker Verteidiger,Position_Rechter Verteidiger,ligacountry_australia,ligacountry_belgium,ligacountry_brazil,ligacountry_china,ligacountry_costa rica,ligacountry_ecuador,ligacountry_england,ligacountry_france,ligacountry_germany,ligacountry_greece,ligacountry_honduras,ligacountry_iran,ligacountry_israeli,ligacountry_italy,ligacountry_korea,ligacountry_mexico,ligacountry_netherlands,ligacountry_norway,ligacountry_portugal,ligacountry_qatar,ligacountry_russia,ligacountry_scotland,ligacountry_spain,ligacountry_switzerland,ligacountry_tunisia,ligacountry_turkey,ligacountry_ukraine,ligacountry_unitedarabianemirtaes,ligacountry_usa,prediction,reality
0,COL,Juan ZUNIGA,SSC Neapel,0.005556,0.002778,0.002778,0.000000,0.000000,0.005556,0.000000,0.002778,0.563889,0.416667,0.008333,0.002778,0.000000,0.094444,0.047222,0.000000,0.000000,0.0,0.101389,0.032778,0.033889,30.5,0.000000,0.000000,0.0,0.011111,0.038889,0.000000,0.000000,0.002778,0.008333,100.0,0.0,0.000000,0.047222,0.000000,0.000000,0.000000,0.000000,0.000000,0.043860,0.032895,0.065789,0.000000,0.000000,0.065789,0.122703,0.126947,0.057692,0.000000,0.267296,0.000000,0.000000,0.000000,0.0,0.054945,0.208333,0.000000,0.000000,0.000000e+00,0.000000,0.066964,0.0,0.000000,72.000000,28.0,13000000.0,11000000.0,-2000000.0,500000.0,Colombia,3,9,15,0,0,0,15,9,3,-0.153846,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.483333e+06,11000000.0
1,COL,Cristian ZAPATA,AC Mailand,0.005556,0.002778,0.002778,0.000000,0.005556,0.000000,0.002778,0.000000,0.375000,0.300000,0.000000,0.000000,0.000000,0.011111,0.002778,0.000000,0.000000,0.0,0.102500,0.030278,0.034444,31.4,0.000000,0.000000,0.0,0.011111,0.011111,0

In [57]:
from sklearn.externals import joblib
joblib.dump(regr, 'abwehr2.pkl') 

['abwehr2.pkl']

In [16]:
testdata_ML_X

array([[0.00277778, 0.        , 0.00277778, ..., 0.        , 0.        ,
        0.        ],
       [0.00740741, 0.        , 0.00740741, ..., 0.        , 0.        ,
        0.        ],
       [0.00769231, 0.00512821, 0.0025641 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.0045045 , 0.0045045 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00257069, 0.00257069, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00512821, 0.0025641 , 0.0025641 , ..., 0.        , 0.        ,
        0.        ]])